<a href="https://colab.research.google.com/github/AffanA2003/20-ntu-cs-1091/blob/main/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers
import os
import cv2
import numpy as np
import shutil

In [2]:
dataset_csv = "dataset.csv"
dataframe = pd.read_csv(dataset_csv)

In [3]:
image_paths = dataframe['filename'].values
widths = dataframe['width'].values
heights = dataframe['height'].values
x_coordinates = dataframe['region_shape_attributes'].apply(lambda x: eval(x)['x']).values
y_coordinates = dataframe['region_shape_attributes'].apply(lambda x: eval(x)['y']).values
widths = dataframe['region_shape_attributes'].apply(lambda x: eval(x)['width']).values
heights = dataframe['region_shape_attributes'].apply(lambda x: eval(x)['height']).values

In [4]:


def copy_images(directory):
    # Create a new directory for saving the copied images
    output_directory = directory + "_copy"
    os.makedirs(output_directory, exist_ok=True)

    # Get the list of image files in the directory
    image_files = [file for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".png"))]

    # Copy each image to the output directory
    for image_file in image_files:
        # Source and destination paths
        source_path = os.path.join(directory, image_file)
        destination_path = os.path.join(output_directory, image_file)
        
        # Copy the image file
        shutil.copy2(source_path, destination_path)

    print("Image copying completed. Copied images are saved in:", output_directory)

# Example usage



In [5]:
image_directory = "images/"
copy_images(image_directory)

Image copying completed. Copied images are saved in: images/_copy


In [6]:
def load_images_from_directory(directory):
    image_files = [file for file in os.listdir(directory) if file.endswith((".jpg", ".jpeg", ".png"))]
    images = []
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = cv2.imread(image_path)
        images.append(image)
    return images

In [7]:
def load_labels_from_csv(csv_file):
    dataframe = pd.read_csv(csv_file)
    labels = dataframe['region_attributes'].apply(lambda x: eval(x)['status']).values
    return labels


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, image_directory, csv_file):
        # Load images and labels from the directory and CSV file
        # Implement your logic here

    def __len__(self):
        # Return the total number of samples in the dataset
        # Implement your logic here

    def __getitem__(self, index):
        # Return a sample from the dataset based on the given index
        # Implement your logic here

# Create an instance of your custom dataset
dataset = CustomDataset(image_directory, csv_file)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define your model architecture
model = nn.Sequential(
    # Add your desired layers to the model
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    # ...
    # Modify and add more layers as per your requirements
    nn.Flatten(),
    nn.Linear(32 * height * width, 1),
    nn.Sigmoid()
)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    train_loss = 0.0
    val_loss = 0.0

    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            val_loss += loss.item() * images.size(0)

    train_loss /= len(train_dataset)
    val_loss /= len(val_dataset)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "path/to/save/model.pth")
